1. Импорт модулей

In [7]:
import sys
import os

sys.path.append(os.path.abspath(".."))

from model import LinearClassifier
from data_utils import get_2d_classification_data
from train import train_model

import torch
import torch.nn as nn
import torch.optim as optim

2. Подготовка данных и модели

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LinearClassifier().to(device)
dataloader = get_2d_classification_data(batch_size=32)

loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

AttributeError: module 'torch.nn' has no attribute 'RelU'